In [ ]:
!pip install -qU langchain langchain-pinecone pinecone-notebooks langchain-google-genai langchain-core langchain_community docx2txt pypdf sentence_transformers

# Step 1: Initialize Pinecone and Set Up API Keys

In [ ]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata

pinecone_api_key = userdata.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
import time

index_name = "basic-rag-system"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

# Step 2: Use LangChain for RAG Workflow

## 1. Set Up Embedding Model

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

## 2. Set Up Document Loader and Text Spliter

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document

In [ ]:
# Function to load documents from a folder

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
      file_path = os.path.join(folder_path, filename)
      if filename.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
      elif filename.endswith(".docx"):
        loader = Docx2txtLoader(file_path)
      else:
        print(f"Unsupported file type: {filename}")
        continue
      documents.extend(loader.load())
    return documents

# Load documents from a folder
folder_path = "/content/docs"
documents = load_documents(folder_path)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 40,
    separators = ["\n\n", "\n", " ", ""],
    length_function = len
)


print(f"Loaded {len(documents)} documents from the folder.")
splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")

Loaded 41 documents from the folder.
Split the documents into 41 chunks.


## 3. Setup Vector Store, Embed and Store Documents.

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(splits))]

In [ ]:
vector_store.add_documents(documents=splits, ids=uuids)

## 4. Set Up Data Retrieval

In [ ]:
# Data Retrieval

results = vector_store.similarity_search(
    "What is the difference between fine-tuning and trasfer learning?",
    k=2,
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Transfer Learning Fine-Tuning
Uses a pre-trained model for new tasks. Adapts a pre-trained model to a specific task.
Generalized knowledge transfer. Task-specific performance improvement.
Retrains only the final layers. Updates specific or all model weights.
Minimal new data. Requires labeled task data.
Limited customization. Fully customizable.
Lower computational cost, as fewer parameters are trained. Higher cost, depending on task complexity.
(c) 2024 Data Science Dojo. No reproduction permitted.
Fine Tuning vs. Transfer Learning
How are they different? [{'page': 11.0, 'source': '/content/docs/Updated Fine-tuning .pdf'}]
* LLM Bootamp
(c) 2024 Data Science Dojo. No reproduction permitted.
Fine Tuning vs.
Transfer Learning
Transfer learning
Fine tuning
Key differences between fine tunign and
transfer learning [{'page': 5.0, 'source': '/content/docs/Updated Fine-tuning .pdf'}]


# Step 3: Set Up Google Gemini Flash Model

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# Step 4: Combine Retriever and LLM

In [ ]:
def answer_to_user(query: str):

  #Vector Search
  vector_results = vector_store.similarity_search(query, k=2)

  final_answer = llm.invoke(f"ANSWER THIS USER QUERY: {query} USING THIS CONTEXT: {vector_results}")

  return final_answer

# Step 5: Query the RAG System

In [ ]:
answer = answer_to_user("What is Fine tuning?")
answer.content

"Fine tuning is a process that unfreezes and retrains some or all layers of a pre-trained model.  This adjusts the model to a specific dataset and its features.  It's used when a moderate to large dataset is available, the task differs from the pre-trained model's original task, and higher customization is needed."